In [11]:
! pip install accelerate

## 학습 데이터 CSV 저장

In [13]:
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from torch.utils.data import Dataset
import pandas as pd
import torch

class ABSADataset(Dataset):
    def __init__(self, data, tokenizer, max_len=128):
        self.data = data
        self.tokenizer = tokenizer
        self.max_len = max_len
        self.label_map = {'positive': 0, 'neutral': 1, 'negative': 2}

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        text = row['sentence']
        aspect = row['aspect']
        label = self.label_map[row['sentiment']]

        encoded = self.tokenizer(
            aspect,
            text,
            padding='max_length',
            truncation=True,
            max_length=self.max_len,
            return_tensors='pt'
        )

        return {
            'input_ids': encoded['input_ids'].squeeze(),
            'attention_mask': encoded['attention_mask'].squeeze(),
            'labels': torch.tensor(label)
        }

# 데이터 불러오기
train_df = pd.read_csv("absa_train.csv")
dev_df = pd.read_csv("absa_dev.csv")

# 모델 & 토크나이저
tokenizer = BertTokenizer.from_pretrained("klue/bert-base")
model = BertForSequenceClassification.from_pretrained("klue/bert-base", num_labels=3)

# 데이터셋 생성
train_dataset = ABSADataset(train_df, tokenizer)
dev_dataset = ABSADataset(dev_df, tokenizer)

# 학습 설정
training_args = TrainingArguments(
    output_dir='./absa_model',
    num_train_epochs=2,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_dir='./logs',
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss"
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=dev_dataset
)

# 학습 시작
trainer.train()

# 모델 저장
model.save_pretrained("./absa_model")
tokenizer.save_pretrained("./absa_model")


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at klue/bert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
d:\Anaconda\envs\sesac\lib\site-packages\transformers\training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


ImportError: Using the `Trainer` with `PyTorch` requires `accelerate>=0.26.0`: Please run `pip install transformers[torch]` or `pip install 'accelerate>=0.26.0'`